# Imports

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import ctypes

In [2]:
lib_path = "./../../mylib/target/debug/libmylib.so"
my_lib = ctypes.cdll.LoadLibrary(lib_path)

## --------------------------init_linear_model--------------------------
# pub extern "C" fn init_linear_model(input_count: u32) -> *mut LinearModel;
my_lib.init_linear_model.argtypes = [ctypes.c_uint32]
my_lib.init_linear_model.restypes = None

# pub extern "C" fn train_linear_model(model: *mut LinearModel, features: *const c_float, labels: *const c_float, data_size: u32, learning_rate: f32, epochs: u32)
my_lib.train_linear_model.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float), ctypes.c_uint32, ctypes.c_float, ctypes.c_uint32]
my_lib.train_linear_model.restype = None

# pub extern "C" fn predict_linear_model(model: *mut LinearModel, inputs: *mut f32) -> c_float
my_lib.predict_linear_model.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float)]
my_lib.predict_linear_model.restype = ctypes.c_float

# pub extern "C" fn free_linear_model(model: *mut LinearModel)
my_lib.free_linear_model.argtypes = [ctypes.c_void_p]
my_lib.free_linear_model.restype = None

## -------------------------- init mlp --------------------------
# pub extern "C" fn init_mlp(npl: *mut u32, npl_size: u32) -> *mut MultiLayerPerceptron 
my_lib.init_mlp.argtypes = [ctypes.POINTER(ctypes.c_uint32), ctypes.c_uint32]
my_lib.init_mlp.restype = ctypes.c_void_p

# pub extern "C" fn train_mlp( model: *mut MultiLayerPerceptron, inputs: *mut c_float, outputs: *mut c_float, row: u32, alpha: c_float, nb_iteration: u32, is_classification: bool)
my_lib.train_mlp.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float), ctypes.c_uint32, ctypes.c_float, ctypes.c_uint32, ctypes.c_bool]
my_lib.train_mlp.restype = None

# pub extern "C" fn predict_mlp(model: *mut MultiLayerPerceptron,sample_inputs: *mut f32,sample_inputs_size: usize,is_classification: bool,) -> *mut f32
my_lib.predict_mlp.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float), ctypes.c_uint32, ctypes.c_bool]
my_lib.predict_mlp.restype = ctypes.POINTER(ctypes.c_float)

# pub extern "C" fn free_mlp(model: *mut MultiLayerPerceptron)
my_lib.free_mlp.argtypes = [ctypes.c_void_p]
my_lib.free_mlp.restype = None


# Cas de tests, rendu 2

## Classification

### Linear Simple :
        Linear Model : OK
        MLP (2, 1)   : OK

In [3]:
# X = np.array([
#     [1.0, 1.0],
#     [2.0, 3.0],
#     [3.0, 3.0]
# ], dtype=np.float32)
# Y = np.array([
#     1.0,
#     -1.0,
#     -1.0
# ], dtype=np.float32)
# plt.scatter(X[0, 0], X[0, 1], color='blue')
# plt.scatter(X[1:3,0], X[1:3,1], color='red')
# plt.show()
# plt.clf()

In [4]:
#init linear model

X = np.array([
    [1.0, 1.0],
    [2.0, 3.0],
    [3.0, 3.0]
], dtype=np.float32)
Y = np.array([
    1.0,
    -1.0,
    -1.0
], dtype=np.float32)
X_flat = X.flatten().astype(ctypes.c_float)
Y_flat = Y.flatten().astype(ctypes.c_float)
X_flat_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_flat_ptr = Y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
data_size = len(Y)

linear_model = my_lib.init_linear_model(2)
# train linear model
my_lib.train_linear_model(linear_model, X_flat_ptr, Y_flat_ptr, data_size, 0.001, 100000)
test_input = np.array([1.0, 1.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_linear_model(linear_model, test_input_p)
print(prediction)

test_input = np.array([2.0, 3.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_linear_model(linear_model, test_input_p)
print(prediction)

test_input = np.array([3.0, 3.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_linear_model(linear_model, test_input_p)
print(prediction)
my_lib.free_linear_model(linear_model)


In [4]:
#init mlp
X = np.array([
    [1.0, 1.0],
    [2.0, 3.0],
    [3.0, 3.0]
], dtype=np.float32)
Y = np.array([
    1.0,
    -1.0,
    -1.0
], dtype=np.float32)
X_flat = X.flatten().astype(ctypes.c_float)
Y_flat = Y.flatten().astype(ctypes.c_float)
X_flat_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_flat_ptr = Y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

npl = np.array([2, 1], dtype=ctypes.c_uint32)
raw_npl = np.ctypeslib.as_ctypes(npl)
mlp_model = my_lib.init_mlp(raw_npl, len(npl))
# Train mlp
my_lib.train_mlp(mlp_model, X_flat_ptr, Y_flat_ptr, 3, 0.001, 100000, ctypes.c_bool(True))
# predict mlp
test_input = np.array([1.0, 1.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_mlp(mlp_model, test_input_p, 2, True)
print(prediction[0])


test_input = np.array([2.0, 3.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_mlp(mlp_model, test_input_p, 2, True)
print(prediction[0])

test_input = np.array([3.0, 3.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_mlp(mlp_model, test_input_p, 2, True)
print(prediction[0])
my_lib.free_mlp(mlp_model)

### Linear Multiple :
        Linear Model : OK
        MLP (2, 1)   : OK

In [8]:
X = np.concatenate([np.random.random((50,2)) * 0.9 + np.array([1, 1]), np.random.random((50,2)) * 0.9 + np.array([2, 2])])
Y = np.concatenate([np.ones((50, 1)), np.ones((50, 1)) * -1.0])
plt.scatter(X[0:50, 0], X[0:50, 1], color='blue')
plt.scatter(X[50:100,0], X[50:100,1], color='red')
plt.show()
plt.clf()

In [6]:
# linear model
X = np.concatenate([np.random.random((50,2)) * 0.9 + np.array([1, 1]), np.random.random((50,2)) * 0.9 + np.array([2, 2])])
Y = np.concatenate([np.ones((50, 1)), np.ones((50, 1)) * -1.0])
X_flat = X.flatten().astype(ctypes.c_float)
Y_flat = Y.flatten().astype(ctypes.c_float)
X_flat_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_flat_ptr = Y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
data_size = len(Y)

#init model
model = my_lib.init_linear_model(2)
# train model
my_lib.train_linear_model(model, X_flat_ptr, Y_flat_ptr, data_size, 0.001, 1000)
# test predict

test_input = np.array([3.0, 3.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_linear_model(model, test_input_p)
print(prediction)

my_lib.free_linear_model(model)

In [7]:
# mlp
X = np.concatenate([np.random.random((50,2)) * 0.9 + np.array([1, 1]), np.random.random((50,2)) * 0.9 + np.array([2, 2])])
Y = np.concatenate([np.ones((50, 1)), np.ones((50, 1)) * -1.0])
X_flat = X.flatten().astype(ctypes.c_float)
Y_flat = Y.flatten().astype(ctypes.c_float)
X_flat_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_flat_ptr = Y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
data_size = len(Y)

npl = np.array([2, 1], dtype=ctypes.c_uint32)
raw_npl = np.ctypeslib.as_ctypes(npl)
mlp_model = my_lib.init_mlp(raw_npl, len(npl))
# Train mlp
my_lib.train_mlp(mlp_model, X_flat_ptr, Y_flat_ptr, data_size, 0.001, 100000, ctypes.c_bool(True))
# predict mlp
test_input = np.array([3.0, 3.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_mlp(mlp_model, test_input_p, 2, True)
print(prediction[0])
my_lib.free_mlp(mlp_model)

### XOR :
        Linear Model    : KO
        MLP (2, 2, 1)   : OK

In [ ]:
# X = np.array([[1, 0], [0, 1], [0, 0], [1, 1]])
# Y = np.array([1, 1, -1, -1])
# plt.scatter(X[0:2, 0], X[0:2, 1], color='blue')
# plt.scatter(X[2:4,0], X[2:4,1], color='red')
# plt.show()
# plt.clf()

In [8]:
X = np.array([[1, 0], [0, 1], [0, 0], [1, 1]])
Y = np.array([1, 1, -1, -1])
X_flat = X.flatten().astype(ctypes.c_float)
Y_flat = Y.flatten().astype(ctypes.c_float)
X_flat_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_flat_ptr = Y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
data_size = len(Y)

linear_model = my_lib.init_linear_model(2)
# train linear model
my_lib.train_linear_model(linear_model, X_flat_ptr, Y_flat_ptr, data_size, 0.001, 100000)
test_input = np.array([3.0, 3.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_linear_model(linear_model, test_input_p)
print(prediction)
my_lib.free_linear_model(linear_model)


In [9]:
X = np.array([[1, 0], [0, 1], [0, 0], [1, 1]])
Y = np.array([1, 1, -1, -1])
X_flat = X.flatten().astype(ctypes.c_float)
Y_flat = Y.flatten().astype(ctypes.c_float)
X_flat_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_flat_ptr = Y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
data_size = len(Y)

npl = np.array([2, 2, 1], dtype=ctypes.c_uint32)
raw_npl = np.ctypeslib.as_ctypes(npl)
mlp_model = my_lib.init_mlp(raw_npl, len(npl))
# Train mlp
my_lib.train_mlp(mlp_model, X_flat_ptr, Y_flat_ptr, data_size, 0.001, 20000000, ctypes.c_bool(True))
# predict mlp
test_input = np.array([1.0, 1.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_mlp(mlp_model, test_input_p, 2, True)
print(prediction[0])


test_input = np.array([1.0, 0.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_mlp(mlp_model, test_input_p, 2, True)
print(prediction[0])


test_input = np.array([0.0, 1.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_mlp(mlp_model, test_input_p, 2, True)
print(prediction[0])


test_input = np.array([0.0, 0.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_mlp(mlp_model, test_input_p, 2, True)
print(prediction[0])

my_lib.free_mlp(mlp_model)

### Cross :
        Linear Model    : KO
        MLP (2, 4, 1)   : OK

In [9]:
X = np.random.random((500, 2)) * 2.0 - 1.0
Y = np.array([1 if abs(p[0]) <= 0.3 or abs(p[1]) <= 0.3 else -1 for p in X])
print("x", X)
print("x", Y)
# plt.scatter(np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]] == 1, enumerate(X)))))[:,0], np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]] == 1, enumerate(X)))))[:,1], color='blue')
# plt.scatter(np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]] == -1, enumerate(X)))))[:,0], np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]] == -1, enumerate(X)))))[:,1], color='red')
# plt.show()
# plt.clf()

In [ ]:
X = np.random.random((500, 2)) * 2.0 - 1.0
Y = np.array([1 if abs(p[0]) <= 0.3 or abs(p[1]) <= 0.3 else -1 for p in X])
X = np.array([[1, 0], [0, 1], [0, 0], [1, 1]])
Y = np.array([1, 1, -1, -1])
X_flat = X.flatten().astype(ctypes.c_float)
Y_flat = Y.flatten().astype(ctypes.c_float)
X_flat_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_flat_ptr = Y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
data_size = len(Y)

linear_model = my_lib.init_linear_model(2)
# train linear model
my_lib.train_linear_model(linear_model, X_flat_ptr, Y_flat_ptr, data_size, 0.001, 100000)
test_input = np.array([3.0, 3.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_linear_model(linear_model, test_input_p)
print(prediction)
my_lib.free_linear_model(linear_model)


In [ ]:
X = np.random.random((500, 2)) * 2.0 - 1.0
Y = np.array([1 if abs(p[0]) <= 0.3 or abs(p[1]) <= 0.3 else -1 for p in X])
X_flat = X.flatten().astype(ctypes.c_float)
Y_flat = Y.flatten().astype(ctypes.c_float)
X_flat_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_flat_ptr = Y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
data_size = len(Y)

npl = np.array([2, 4, 1], dtype=ctypes.c_uint32)
raw_npl = np.ctypeslib.as_ctypes(npl)
mlp_model = my_lib.init_mlp(raw_npl, len(npl))
# Train mlp
my_lib.train_mlp(mlp_model, X_flat_ptr, Y_flat_ptr, data_size, 0.001, 20000000, ctypes.c_bool(True))
# predict mlp
test_input = np.array([1.0, 0.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_mlp(mlp_model, test_input_p, 2, True)
print(prediction[0])
my_lib.free_mlp(mlp_model)

### Multi Linear 3 classes :
        Linear Model x3 : OK
        MLP (2, 3)      : OK

In [5]:
X = np.random.random((500, 2)) * 2.0 - 1.0
Y = np.array([[1, 0, 0] if -p[0] - p[1] - 0.5 > 0 and p[1] < 0 and p[0] - p[1] - 0.5 < 0 else
              [0, 1, 0] if -p[0] - p[1] - 0.5 < 0 and p[1] > 0 and p[0] - p[1] - 0.5 < 0 else
              [0, 0, 1] if -p[0] - p[1] - 0.5 < 0 and p[1] < 0 and p[0] - p[1] - 0.5 > 0 else
              [0, 0, 0]for p in X])

X = X[[not np.all(arr == [0, 0, 0]) for arr in Y]]
Y = Y[[not np.all(arr == [0, 0, 0]) for arr in Y]]
print("X", X)
print("Y", Y)

In [ ]:
plt.scatter(np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][0] == 1, enumerate(X)))))[:,0], np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][0] == 1, enumerate(X)))))[:,1], color='blue')
plt.scatter(np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][1] == 1, enumerate(X)))))[:,0], np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][1] == 1, enumerate(X)))))[:,1], color='red')
plt.scatter(np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][2] == 1, enumerate(X)))))[:,0], np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][2] == 1, enumerate(X)))))[:,1], color='green')
plt.show()
plt.clf()

### Multi Cross :
        Linear Model x3 : KO
        MLP (2, ?, ?, 3): OK

In [6]:
X = np.random.random((1000, 2)) * 2.0 - 1.0
Y = np.array([[1, 0, 0] if abs(p[0] % 0.5) <= 0.25 and abs(p[1] % 0.5) > 0.25 else [0, 1, 0] if abs(p[0] % 0.5) > 0.25 and abs(p[1] % 0.5) <= 0.25 else [0, 0, 1] for p in X])

In [ ]:
plt.scatter(np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][0] == 1, enumerate(X)))))[:,0], np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][0] == 1, enumerate(X)))))[:,1], color='blue')
plt.scatter(np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][1] == 1, enumerate(X)))))[:,0], np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][1] == 1, enumerate(X)))))[:,1], color='red')
plt.scatter(np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][2] == 1, enumerate(X)))))[:,0], np.array(list(map(lambda elt : elt[1], filter(lambda c: Y[c[0]][2] == 1, enumerate(X)))))[:,1], color='green')
plt.show()
plt.clf()

## Régression

### Linear Simple 2D :
        Linear Model : OK
        MLP (1, 1)   : OK

In [4]:
X = np.array([
      [1],
      [2]
])
Y = np.array([
      2,
      3
])

In [5]:
# model linear
X_flat = X.flatten().astype(ctypes.c_float)
Y_flat = Y.flatten().astype(ctypes.c_float)
X_flat_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_flat_ptr = Y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
data_size = len(Y)

#init model
model = my_lib.init_linear_model(1)
# train model
my_lib.train_linear_model(model, X_flat_ptr, Y_flat_ptr, data_size, 0.001, 100000)
# test predict

test_input = np.array([1.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_linear_model(model, test_input_p)
print(prediction)

test_input = np.array([2.0], dtype=ctypes.c_float)
test_input_p = np.ctypeslib.as_ctypes(test_input)
prediction = my_lib.predict_linear_model(model, test_input_p)
print(prediction)

my_lib.free_linear_model(model)

### Non Linear Simple 2D :
        Linear Model    : OK
        MLP (1, ?, 1)   : OK

In [ ]:
X = np.array([
      [1],
      [2],
      [3]
])
Y = np.array([
      2,
      3,
      2.5
])

In [ ]:
plt.scatter(X,Y)
plt.show()
plt.clf()

### Linear Simple 3D :
        Linear Model    : OK
        MLP (2, 1)      : OK

In [ ]:
X = np.array([
      [1, 1],
      [2, 2],
      [3, 1]
])
Y = np.array([
      2,
      3,
      2.5
])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(X[:,0],X[:,1],Y)
plt.show()
plt.clf()

### Linear Tricky 3D :
        Linear Model    : OK
        MLP (2, 1)      : OK

In [ ]:
X = np.array([
      [1, 1],
      [2, 2],
      [3, 3]
])
Y = np.array([
      1,
      2,
      3
])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(X[:,0],X[:,1],Y)
plt.show()
plt.clf()

### Non Linear Simple 3D :
        Linear Model       : KO
        MLP (2, 2, 1)      : OK

In [ ]:
X = np.array([
      [1, 0],
      [0, 1],
      [1, 1],
      [0, 0],
])
Y = np.array([
      2,
      1,
      -2,
      -1
])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(X[:,0],X[:,1],Y)
plt.show()
plt.clf()